# 네이버 블로그 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
#from selenium import webdriver

In [2]:
import time
import random
import datetime
import re
#import pickle

## blog URL

In [3]:
def get_blogs(url):
    results = []
    urls = []
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    lis = soup.find_all('li', attrs={'class':'sh_blog_top', 'id':re.compile('sp_blog')})
    
    for li in lis:

        title = li.find('a', attrs={'class':re.compile(r'_sp_each_title')}).get('title').strip()
        if li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).text:
            naver_url = li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).get('href').strip()
#        pub = li.find('span', attrs={'class':'_sp_each_source'}).text.strip()
#        date = li.find('span', attrs={'class':'bar'}).next_sibling.strip()
        results.append([title, naver_url])
        urls.append(naver_url)
        
    return urls

## real data URL

In [4]:
def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url   

## Title

In [5]:
def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

## Datetime

In [6]:
def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

## Text

In [7]:
def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).find('div'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts

##  naver URL 구성
* where = post(블로그)<br/>
* query = <br/>
* st = sim(관련도순), date(최신순) <br/>
* date_from = <br/>
* date_to = <br/>
* date_option = 0(전체), 2,3,4,5,6,7(1일,1주,1개월,6개월,1년), 8(직접입력) <br/>
* srchby = all(전체), title(제목)<br/>

## selenium으로 blog url 불러오기

In [8]:
from tqdm import tqdm_notebook
import pandas as pd

In [ ]:
# 네이버 클로바 출시일- 2017.10
# skt 누구 - 2016.09
# kt 기가지니 - 2017.01
# 카카오 미니 - 2017.11

In [9]:
# 출시일 한달 전부터 2019-04-30까지
date_pd = pd.date_range('2017-10-01', '2019-04-30').strftime("%Y%m%d")
date_pd

Index(['20171001', '20171002', '20171003', '20171004', '20171005', '20171006',
       '20171007', '20171008', '20171009', '20171010',
       ...
       '20190421', '20190422', '20190423', '20190424', '20190425', '20190426',
       '20190427', '20190428', '20190429', '20190430'],
      dtype='object', length=577)

In [10]:
query = '"카카오%20미니"%20스피커'

#pickle_name = q+'_'+startDate+'_'+endDate+'.p'
total_url = []

for date in tqdm_notebook(date_pd):
    for page in range(1,100):
        try:
            time.sleep(0.01)

            url = 'https://search.naver.com/search.naver?date_from='+str(date)+'&date_option=8&date_to='+str(date)+'&dup_remove=1&nso=p%3Afrom'+str(date)+'to'+str(date)+'&query='+query+'&where=post&start='+str(page)

            results = get_blogs(url)

            if results[0] in total_url:
                break

            total_url = total_url + results
        except:
            break     

#pickle.dump(total_results.open(pickle_name,'wb'))

In [13]:
len(set(total_url))

2983

In [14]:
len(total_url)

2992

In [11]:
total_url[257]

'http://blog.mzzang.net/blog/51895'

In [12]:
total_url.remove(total_url[257])

In [48]:
total_url[1156]

'https://blog.dreamyoungs.com/2018/03/09/%EA%BF%88%EB%A7%8E%EC%9D%80%EC%B2%AD%EB%85%84%EB%93%A4-x-%EC%B9%B4%EC%B9%B4%EC%98%A4-%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4%EA%B3%B5%EC%8B%9D-%EC%97%90%EC%9D%B4%EC%A0%84%EC%8B%9C%EC%82%AC-%EC%84%A0/'

In [46]:
real_url = get_realdata_url_naverblog(total_url[257])
r = requests.get(real_url)
soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")
print(get_texts_naverblog(soup))
print(get_title_naverblog(soup))
print(det_publish_datetime_naverblog(soup))


NA
NA
NA


## Get real_url, title, datetime, text

In [13]:
with open('kakaomini_speaker.txt', 'w', encoding = 'utf8') as f:
    title_list = []
    publish_datetime_list = []
    text_list = []
    url_list = []
    
    for full_url in tqdm_notebook(total_url):
        
        try:       
            # 실제 데이터가 들어있는 url 추출
            real_url = get_realdata_url_naverblog(full_url)
            r = requests.get(real_url)
            soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

            # 블로그 포스트 제목 추출
            title = get_title_naverblog(soup)
            if title in title_list:
                continue
            title_list.append(title)

            # 블로그 작성일 추출
            publish_datetime = det_publish_datetime_naverblog(soup)
            publish_datetime_list.append(publish_datetime)

            # 블로그 텍스트 추출
            texts = get_texts_naverblog(soup)
            text_list.append(texts)

            # 중복 제거된 url
            url_list.append(real_url)

            #content = ' '.join(texts)
            f.write(texts+'\n') #결과를 text 파일에 저장합니다.
            
        except:
            continue

In [15]:
len(url_list)

2588

In [16]:
title_list[:10]

['(스크랩)로엔엔터테인먼트의 멜론, 인공지능 스피커 덕에 급성장 확실',
 '대세는 인공지능 스피커! 네이버 웨이브 VS 카카오미니',
 "[2017' 칸광고제] 소셜 플랫폼의 새로운 진화. 페이스북 챗봇(Facebook Chatbot)을 활용한 브라질 10대 청소년들의 음주 예방/치료 캠페인 - Alcoholics Anonymous (AA) 'Anonymous Friend' (2017' 칸광고제/페이스북 어워즈 수상작)",
 '공감수 순위 목록 2017년 10월 02일 기준 ',
 '조회수 순위 목록 2017년 10월 02일 기준  ',
 '카카오미니 완판!! 얼마나 좋길래??',
 '조회수 순위 목록 2017년 10월 03일 기준 ',
 '공감수 순위 목록 2017년 10월 03일 기준 ',
 "누구? 아리아? 모두가 헷갈리는 AI 스피커의 '이름들' 한방에 정리해 줄게",
 '[지나툰] 2017.09.18 :: 카카오미니']

In [18]:
print(text_list[1])

안녕하세요. 에스메모, 앱 플레이 미뮤 등 다양한 IT Life 플랫폼 퍼블리싱 기업 민앤지 매니저S입니다.  얼마 전 실시간 검색 1위에 오른 ‘카카오미니’ 덕분에 인공지능 스피커에 대한 관심이 더욱 더 커졌을 것으로 보입니다! 아직까지는 우리에게 낯설기도 한 인공지능 스피커, 과연 어떤 것이고 최근 출시된 네이버 웨이브와 카카오미니는 어떤 기능을 가지고 있는지에 대해 한 번 알아 볼게요.
스마트폰의 개인비서로 쓰이고 있는 인공지능은 이제 그 경쟁의 장을 블루투스 스피커로 확장시킨 상태입니다. 지난 2014년 11월 아마존닷컴이 처음 선보인 ‘에코’를 시작으로 전 세계의 IT 기업들이 경쟁에 참여하는 추세인데요. 최근에는 애플에서도 시리가 탑재된 인공지능 스피커 ‘홈팟’을 공개하며 화제를 불러 오기도 했죠.
물론 국내에서도 다양한 제품들이 시장에 출두하고 있습니다. 이동통신 3사를 중심으로 ‘누구’와 ‘기가지니’가 그 동안 초기 시장을 선점한 상태라면, 이제 막 모습을 드러낸 ‘카카오미니’와 ‘네이버 웨이브’는 경쟁을 더욱 심화시키는 역할을 할 것이라고 예측 가능한데요~ 이처럼 인공지능 스피커에 주요 기업들이 관심을 갖는 까닭은 바로 이들이 곧 다가올 사물인터넷 시대의 허브가 될 것이기 때문입니다. 즉 새로 펼쳐질 시대에서 우위를 점하기 위한 전초전이라고 할 수 있어요!
이 달인 10월 말 출시되는 카카오미니가 어떤 제품인지부터 먼저 알아 볼까요? 카카오미니에는 카카오에서 개발한 AI 음성 인터페이스 ‘Kakao I’가 탑재된다고 하는데요! 카카오톡과 멜론, 다음과 연동되며 음성으로 카카오 관련 서비스를 이용할 수 있습니다. 특히 카카오미니 사용자는 음성으로 카카오톡 메시지를 전송 가능해요~ 이 외에도 알람, 메모, 택시 호출 등 생활에 밀접한 서비스를 제공할 예정이라고 합니다.
인공지능 스피커 카카오미니는 지난 9월 중순 예약 판매를 진행했는데요. 선풍적인 인기로 예약이 시작되자마자 홈페이지가 마비될 정도였다고 해요! 선예약자들에게는 정상가의 50% 가격으

## dataframe.to_csv

In [19]:
import pandas as pd
df = pd.DataFrame(data={'title': title_list, 
                        'publish_datetime': publish_datetime_list,
                        'text': text_list, 
                        'url': url_list})

In [20]:
#df = df[['title', 'publish_datetime', 'text', 'url']]
print(df.shape)
df.head()

(2588, 4)


,title,publish_datetime,text,url
0,"(스크랩)로엔엔터테인먼트의 멜론, 인공지능 스피커 덕에 급성장 확실",2017-10-01 22:17:00,원문보기,http://blog.naver.com/PostView.nhn?blogId=m_co...
1,대세는 인공지능 스피커! 네이버 웨이브 VS 카카오미니,2017-10-02 10:30:00,"안녕하세요. 에스메모, 앱 플레이 미뮤 등 다양한 IT Life 플랫폼 퍼블리싱 기...",http://blog.naver.com/PostView.nhn?blogId=smar...
2,[2017' 칸광고제] 소셜 플랫폼의 새로운 진화. 페이스북 챗봇(Facebook ...,2017-10-02 15:36:00,NA,http://blog.naver.com/PostView.nhn?blogId=stus...
3,공감수 순위 목록 2017년 10월 02일 기준,2017-10-03 09:44:00,NA,http://blog.naver.com/PostView.nhn?blogId=medo...
4,조회수 순위 목록 2017년 10월 02일 기준,2017-10-03 09:46:00,NA,http://blog.naver.com/PostView.nhn?blogId=7337...


In [31]:
df.to_csv('./kakaomini_speaker_blog.csv',sep=',',encoding='UTF-8')

In [32]:
pd.read_csv(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\final\S01_data_scraping\naver_blog\kakaomini_speaker_blog.csv', encoding='UTF-8',index_col=0)

,title,publish_datetime,text,url
0,"(스크랩)로엔엔터테인먼트의 멜론, 인공지능 스피커 덕에 급성장 확실",2017-10-01 22:17:00,원문보기,http://blog.naver.com/PostView.nhn?blogId=m_co...
1,대세는 인공지능 스피커! 네이버 웨이브 VS 카카오미니,2017-10-02 10:30:00,"안녕하세요. 에스메모, 앱 플레이 미뮤 등 다양한 IT Life 플랫폼 퍼블리싱 기...",http://blog.naver.com/PostView.nhn?blogId=smar...
2,[2017' 칸광고제] 소셜 플랫폼의 새로운 진화. 페이스북 챗봇(Facebook ...,2017-10-02 15:36:00,NaN,http://blog.naver.com/PostView.nhn?blogId=stus...
3,공감수 순위 목록 2017년 10월 02일 기준,2017-10-03 09:44:00,NaN,http://blog.naver.com/PostView.nhn?blogId=medo...
4,조회수 순위 목록 2017년 10월 02일 기준,2017-10-03 09:46:00,NaN,http://blog.naver.com/PostView.nhn?blogId=7337...
5,카카오미니 완판!! 얼마나 좋길래??,2017-10-04 13:30:00,안녕하세요~~오늘은 엄청 핫했던!!카카오미니에 대한 포스팅입니다^^!!이번에 9월 ...,http://blog.naver.com/PostView.nhn?blogId=alse...
6,조회수 순위 목록 2017년 10월 03일 기준,2017-10-04 07:29:00,NaN,http://blog.naver.com/PostView.nhn?blogId=7337...
7,공감수 순위 목록 2017년 10월 03일 기준,2017-10-04 07:31:00,NaN,http://blog.naver.com/PostView.nhn?blogId=7337...
8,누구? 아리아? 모두가 헷갈리는 AI 스피커의 '이름들' 한방에 정리해 줄게,2017-10-05 17:24:00,"최근 IT 대기업들이 AI 스피커 출시에 박차를 가하고 있다. 8, 9월에만 해도 ...",http://blog.naver.com/PostView.nhn?blogId=dohk...
9,[지나툰] 2017.09.18 :: 카카오미니,2017-10-06 23:49:00,NaN,http://blog.naver.com/PostView.nhn?blogId=jina...
